<a href="https://colab.research.google.com/github/shivamishu/cmpe255/blob/main/StateOfTheArtLibrariesForApproximateNearestNeighborSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using state of the art libraries for Approximate nearest neighbor**

**Shivam Shrivastav**

#**Implementing the below ANN alogorithms**

1. Exhaustive Search
2. Product Quantization
3. Trees and Graphs
4. LSH: Locality-sensitive hashing
5. HNSW: Hierarchical Navigable Small World

##**Implementing above all using Faiss**

In [ ]:
!pip install faiss

In [ ]:
!pip3 install faiss
!sudo apt-get install libopenblas-dev
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
import faiss
import pickle
import pandas as pd


In [ ]:
!pwd

/content


In [ ]:
!ls


drive  movies.pickle  sample_data


**pickle load function**

In [ ]:
def data_set():
    with open('movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = data_set()
vectors = data["vector"]
names = data["name"]
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

#**Exhaustive Search**

Comparing each point to every other point, which will require Linear query time (the size of the dataset)

In [ ]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = BruteForceIndex(data["vector"], data["name"])
#index.build()

## Checking Similarity

In [ ]:
movie_vector, movie_name = data['vector'][80:91], data['name'][80]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")


The most similar movies to Hudsucker Proxy, The (1994) are:
* Hudsucker Proxy, The (1994)
* Bob Roberts (1992)
* Ed Wood (1994)
* Heathers (1989)
* This Is Spinal Tap (1984)
* Sirens (1994)
* In the Name of the Father (1993)
* Vanya on 42nd Street (1994)
* Quiz Show (1994)
* What's Eating Gilbert Grape (1993)


# **Product Quantization**

Product quantization is an effective vector quantization approach to compactly encode high-dimensional vectors for fast approximate nearest neighbor (ANN) search.

In [ ]:
class PQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = PQIndex(data["vector"], data["name"])
index.build()

## Checking Similarity

In [ ]:
movie_index = 80
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movies to Hudsucker Proxy, The (1994) are:


['Hudsucker Proxy, The (1994)',
 'Bob Roberts (1992)',
 'Secret Garden, The (1993)',
 'Ed Wood (1994)',
 'Bullets Over Broadway (1994)',
 'Nikita (La Femme Nikita) (1990)',
 'Harold and Maude (1971)',
 'Sirens (1994)',
 "Microcosmos: Le peuple de l'herbe (1996)",
 'Fearless (1993)']

# **Trees and Graph**

In [ ]:
!pip install annoy

     |████████████████████████████████| 646 kB 39.0 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391692 sha256=8446334f34f319356fb11f10aef935390304f58c32365c083e9043ea7fb7a50d
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [ ]:
import annoy
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [ ]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # Remove the CWD from sys.path while we load stuff.


## Checking Similarity

In [ ]:
movie_vector, movie_name = data['vector'][70], data['name'][70]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Lion King, The (1994) are:
* Lion King, The (1994)
* Aladdin (1992)
* Snow White and the Seven Dwarfs (1937)
* Beauty and the Beast (1991)
* Dumbo (1941)
* Cinderella (1950)
* Fantasia (1940)
* Sound of Music, The (1965)
* Pinocchio (1940)
* E.T. the Extra-Terrestrial (1982)


#**LSH**
LSH-based algorithms are one of the most common strategies when it comes to ANN. They construct a hash table as their data structure by mapping points that are nearby into the same bucket.

I am going to show how to use faiss, to do “Approximate Nearest Neighbors Using LSH”.

In [ ]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [ ]:
data["vector"]

array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
        -0.17529577, -0.03061521],
       [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
        -0.09166156,  0.05047869],
       [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
        -0.08328608, -0.0152858 ],
       ...,
       [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
         0.14679626,  0.2462584 ],
       [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
         0.16223577,  0.19897607],
       [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
         0.0467432 ,  0.07893164]], dtype=float32)

In [ ]:
index.query(data['vector'])

['Supercop (1992)',
 'Rumble in the Bronx (1995)',
 'Mission: Impossible (1996)',
 'Four Rooms (1995)',
 'Donnie Brasco (1997)',
 'Cold Comfort Farm (1995)',
 'Toy Story (1995)',
 'Angels and Insects (1995)',
 'Twelve Monkeys (1995)',
 'Lone Star (1996)']

#**HNSW**

In [1]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

##Data Charactersitics: 

https://archive.ics.uci.edu/ml/datasets/SIFT10M

In [2]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [3]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [4]:
!ls


sample_data  sift  sift.tar.gz


In [5]:
# data we will search through
wb = read_fvecs('/content/sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [6]:
print (xq)

[[  1.   3.  11. 110.  62.  22.   4.   0.  43.  21.  22.  18.   6.  28.
   64.   9.  11.   1.   0.   0.   1.  40. 101.  21.  20.   2.   4.   2.
    2.   9.  18.  35.   1.   1.   7.  25. 108. 116.  63.   2.   0.   0.
   11.  74.  40. 101. 116.   3.  33.   1.   1.  11.  14.  18. 116. 116.
   68.  12.   5.   4.   2.   2.   9. 102.  17.   3.  10.  18.   8.  15.
   67.  63.  15.   0.  14. 116.  80.   0.   2.  22.  96.  37.  28.  88.
   43.   1.   4.  18. 116.  51.   5.  11.  32.  14.   8.  23.  44.  17.
   12.   9.   0.   0.  19.  37.  85.  18.  16. 104.  22.   6.   2.  26.
   12.  58.  67.  82.  25.  12.   2.   2.  25.  18.   8.   2.  19.  42.
   48.  11.]]


In [7]:
xq.shape

(1, 128)

In [8]:
wb.shape

(1000000, 128)

In [9]:
# 1M samples
xb = read_fvecs('/content/sift/sift_base.fvecs')
# queries
xq = read_fvecs('/content/sift/sift_base.fvecs')[0].reshape(1, -1)
xq_full = read_fvecs('/content/sift/sift_query.fvecs')

In [10]:
!pip install faiss

     |████████████████████████████████| 4.7 MB 7.2 MB/s 


In [11]:
!sudo apt-get install libopenblas-dev
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (228 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot

In [12]:
import faiss

In [13]:
# setup our HNSW parameters
d = 128  # vector size
M = 32
efSearch = 32  # number of entry points (neighbors) we use on each layer
efConstruction = 32  # number of entry points used on each layer
                     # during construction

index = faiss.IndexHNSWFlat(d, M)
print(index.hnsw)

<faiss.swigfaiss.HNSW; proxy of <Swig Object of type 'faiss::HNSW *' at 0x7fc124b0cf00> >


In [14]:
# the HNSW index starts with no levels
index.hnsw.max_level

-1

In [15]:
# and levels (or layers) are empty too
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

**We can set the efConstruction and efSearch parameters, only efConstruction must be set before building the index. efSearch only affects search time behavior.**

In [16]:
index.hnsw.efConstruction = efConstruction
index.hnsw.efSearch = efSearch

In [ ]:
index.add(xb)

In [17]:
# after adding our data we will find that the level
# has been set automatically
index.hnsw.max_level

-1

In [18]:
# and levels (or layers) are now populated
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

In [19]:
index.hnsw.entry_point


-1

**The HNSW::set_default_probas function (from HNSW.cpp)calculates the number of neighbors (in total) a vertex will have across the calculated number of layers. We find that Faiss' implementation does not use M_max or M_max0 directly, but instead uses M to set these values. M_max is set to M, and M_max is set to 2*M**

In [20]:
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbors count = 0
    cum_nneighbor_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbors is == M on every level except level 0 where == M*2
        nn += M*2 if level == 0 else M
        cum_nneighbor_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbor_per_level

In [21]:
assign_probas, cum_nneighbor_per_level = set_default_probas(
    32, 1/np.log(32)
)
assign_probas, cum_nneighbor_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])